重複を検出する。<br>
Idでソートしたときの前後との重複、時間でソートしたときの前後との重複をそれぞれ feature にする。

In [1]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [103]:
CHUNKSIZE = 100000

TRAIN_DATE        = "../../data/train_date.csv"
TRAIN_NUMERIC     = "../../data/train_numeric.csv"
TRAIN_CATEGORICAL = "../../data/train_categorical.csv"
TEST_DATE         = "../../data/test_date.csv"
TEST_NUMERIC      = "../../data/test_numeric.csv"
TEST_CATEGORICAL  = "../../data/test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000
#NROWS = 10000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'

TEST_START_TIME      = "../../data/test_StartEndTime.csv"
TRAIN_START_TIME     = "../../data/train_StartEndTime.csv"

    

In [104]:
chunk_train_date = pd.read_csv(TRAIN_DATE, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_date  = pd.read_csv(TEST_DATE, nrows=NROWS, chunksize=CHUNKSIZE)

df_train_date = chunk_train_date.get_chunk(10)
df_test_date  = chunk_test_date.get_chunk(10)


In [105]:
df_train_date.head()

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
STATION_NUM = 52
station_line_list = [-999 for i in range(STATION_NUM)]
unique_col_list = ['Id']
cols = df_train_date.columns

for column_name in cols:
    if column_name != 'Id':
        r = parse("L{}_S{}_{}", column_name)
        line_id    = int(r[0])
        station_id = int(r[1])
        if (-999 == station_line_list[station_id]):
            station_line_list[station_id] = line_id
            unique_col_list.append(column_name)
        elif (line_id != station_line_list[station_id]):
            print('★★★ERR')

            
    

In [107]:
print(station_line_list)
print(unique_col_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
['Id', 'L0_S0_D1', 'L0_S1_D26', 'L0_S2_D34', 'L0_S3_D70', 'L0_S4_D106', 'L0_S5_D115', 'L0_S6_D120', 'L0_S7_D137', 'L0_S8_D145', 'L0_S9_D152', 'L0_S10_D216', 'L0_S11_D280', 'L0_S12_D331', 'L0_S13_D355', 'L0_S14_D360', 'L0_S15_D395', 'L0_S16_D423', 'L0_S17_D432', 'L0_S18_D437', 'L0_S19_D454', 'L0_S20_D462', 'L0_S21_D469', 'L0_S22_D543', 'L0_S23_D617', 'L1_S24_D677', 'L1_S25_D1854', 'L2_S26_D3037', 'L2_S27_D3130', 'L2_S28_D3223', 'L3_S29_D3316', 'L3_S30_D3496', 'L3_S31_D3836', 'L3_S32_D3852', 'L3_S33_D3856', 'L3_S34_D3875', 'L3_S35_D3886', 'L3_S36_D3919', 'L3_S37_D3942', 'L3_S38_D3953', 'L3_S39_D3966', 'L3_S40_D3981', 'L3_S41_D3997', 'L3_S42_D4029', 'L3_S43_D4062', 'L3_S44_D4101', 'L3_S45_D4125', 'L3_S46_D4135', 'L3_S47_D4140', 'L3_S48_D4194', 'L3_S49_D4208', 'L3_S50_D4242', 'L3_S51_D4255']


In [108]:
df_train_bin = pd.read_csv(TRAIN_DATE, usecols=unique_col_list, nrows=NROWS)
df_test_bin  = pd.read_csv(TEST_DATE,  usecols=unique_col_list, nrows=NROWS)


In [109]:
df_start_train = pd.read_csv(TRAIN_START_TIME, dtype='float32')
df_start_train = df_start_train.ix[:,['Id','StartTime','EndTime', 'Response']]
df_start_test = pd.read_csv(TEST_START_TIME, dtype='float32')
df_start_test['Response'] = -1
df_start_all = pd.concat([df_start_train, df_start_test]).fillna(0) 


C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [110]:
df_start_all.sort_values('Id').head()

,Id,StartTime,EndTime,Response
0,1.0,792.719971,800.700012,-1.0
1,2.0,1025.630005,1060.069946,-1.0
2,3.0,671.919983,711.080017,-1.0
0,4.0,82.239998,87.290001,0.0
3,5.0,255.449997,256.279999,-1.0


In [111]:
df_all = pd.concat([df_train_bin, df_test_bin]).fillna(0)
df_all = pd.merge(df_all, df_start_all, on='Id', how='left')

In [112]:
df_all.sort_values(['StartTime', 'Id']).head(500)

,Id,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S4_D106,L0_S5_D115,L0_S6_D120,L0_S7_D137,L0_S8_D145,...,L3_S45_D4125,L3_S46_D4135,L3_S47_D4140,L3_S48_D4194,L3_S49_D4208,L3_S50_D4242,L3_S51_D4255,StartTime,EndTime,Response
1183777,51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
370,725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1188342,9221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
5806,11549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1191208,15017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
8987,17873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1193226,19054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1194627,21894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
12359,24623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12360,24624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
# ソートしてから、feature が前の列と全く同じものを検出する。
def make_coh_feature(df_org, sort_columns):
    df = df_org.sort_values(sort_columns).reset_index(drop=True)
    
    fname_coh1      = sort_columns[0] + '_coh' 
    fname_coh_prev1 = sort_columns[0] + '_coh_prev' 

    vec_coh      = np.zeros(len(df))
    vec_coh_prev = np.zeros(len(df))

    loopcnt = 0
    for index, row in df.iterrows():
        vec_cur  = row.values[1:1 + STATION_NUM] 
        if 0 < index:             
            if (vec_cur == vec_prev).sum() == STATION_NUM:
                vec_coh[index]        = 1
                vec_coh_prev[index-1] = 1 
        vec_prev = vec_cur

#    df_ret = pd.DataFrame({'Id'           : df['Id'].values, 
#                           fname_coh1     : vec_coh, 
#                           fname_coh_prev1: vec_coh_prev})
    df_ret = pd.DataFrame( columns=['Id', fname_coh1, fname_coh_prev1])
    df_ret.loc[:, 'Id']            = df['Id'].values 
    df_ret.loc[:, fname_coh1]      = vec_coh
    df_ret.loc[:, fname_coh_prev1] = vec_coh_prev
    df_ret = df_ret.sort_values('Id')
    return df_ret


In [114]:
df_all.head(5)

,Id,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S4_D106,L0_S5_D115,L0_S6_D120,L0_S7_D137,L0_S8_D145,...,L3_S45_D4125,L3_S46_D4135,L3_S47_D4140,L3_S48_D4194,L3_S49_D4208,L3_S50_D4242,L3_S51_D4255,StartTime,EndTime,Response
0,4,82.24,82.24,82.24,0.00,82.26,0.00,0.00,82.26,82.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82.239998,87.290001,0.0
1,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1313.119995,1315.750000,0.0
2,7,1618.70,1618.70,1618.70,0.00,0.00,1618.72,1618.72,0.00,1618.73,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1618.699951,1624.420044,0.0
3,9,1149.20,1149.20,1149.21,0.00,1149.22,0.00,0.00,1149.22,1149.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1149.199951,1154.160034,0.0
4,11,602.64,602.64,0.00,602.64,602.66,0.00,0.00,602.67,602.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,602.640015,606.020020,0.0


In [115]:
df_id_dup_list = make_coh_feature(df_all, ['Id'])
df_s_time_dup_list = make_coh_feature(df_all, ['StartTime', 'Id'])
df_e_time_dup_list = make_coh_feature(df_all, ['EndTime', 'Id'])

In [116]:
df_id_dup_list.head(5)

,Id,Id_coh,Id_coh_prev
0,1,0.0,0.0
1,2,0.0,0.0
2,3,0.0,0.0
3,4,0.0,0.0
4,5,0.0,0.0


In [117]:
df_s_time_dup_list.head(5)

,Id,StartTime_coh,StartTime_coh_prev
1101354,1,0.0,0.0
1483509,2,0.0,0.0
886792,3,0.0,0.0
99917,4,0.0,0.0
334310,5,0.0,0.0


In [118]:
df_e_time_dup_list.head(5)

,Id,EndTime_coh,EndTime_coh_prev
1095642,1,0.0,0.0
1482349,2,0.0,0.0
930939,3,0.0,0.0
100182,4,0.0,0.0
333383,5,0.0,0.0


In [119]:
# 保存する。
df_feat = pd.DataFrame(df_all['Id']).sort_values('Id')
df_feat = pd.merge(df_feat, df_id_dup_list, on='Id', how='left')
df_feat = pd.merge(df_feat, df_s_time_dup_list, on='Id', how='left')
df_feat = pd.merge(df_feat, df_e_time_dup_list, on='Id', how='left')





In [120]:
df_feat.head()

,Id,Id_coh,Id_coh_prev,StartTime_coh,StartTime_coh_prev,EndTime_coh,EndTime_coh_prev
0,1,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
df_feat.to_csv("../../data/Feature_Duplicates.csv", index=False)

1
